In [1]:
from openai import OpenAI
import json

client = OpenAI()

# 函数形式的知识库

这个函数可以
* 调用后端接口
* 查询数据库
* 搜索文档库
* 搜索互联网

不过为了方便，这里仅演示硬编码的知识库

In [2]:
def get_current_weather(location):
    if "北京" in location.lower():
        return json.dumps({"location": "北京", "temperature": "-4"}, ensure_ascii=False)
    elif "上海" in location.lower():
        return json.dumps({"location": "上海", "temperature": "1"}, ensure_ascii=False)
    elif "广州" in location.lower():
        return json.dumps({"location": "广州", "temperature": "20"}, ensure_ascii=False)
    else:
        return json.dumps({"location": location, "temperature": "unknown"}, ensure_ascii=False)

# 使上面这个函数被嵌入大模型

In [3]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "获取给定地点的天气",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "地点的名称，例如：上海，北京",
                },
            },
            "required": ["location"],
        },
    },
}]

# 准备问题

In [4]:
messages = [{"role": "user", "content": "我明天想去北京玩，需要随身携带些什么东西？"}]
# messages = [{"role": "user", "content": "我明天想去广州玩，需要随身携带些什么东西？"}]
# messages = [{"role": "user", "content": "我比较怕冷，那么北京、上海、广州三个城市那个更适合我去玩？"}]

# 提问

In [5]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )

AuthenticationError: Error code: 401 - {'error': {'message': 'The OpenAI account associated with this API key has been deactivated. If you are the developer for this OpenAI app, please check your email for more information. If you are seeing this error while using another app or site, please reach out to them for more help.', 'type': 'invalid_request_error', 'param': None, 'code': 'account_deactivated'}}

# 此时GPT会返回一个提示，要求调用知识库

In [95]:
response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hTxXPG3Q7T4uBuCaK828q7S2', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_TWcj0quTMn8MY3qjPYC1o3UR', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function')])

In [96]:
tool_calls = response_message.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_hTxXPG3Q7T4uBuCaK828q7S2', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_TWcj0quTMn8MY3qjPYC1o3UR', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function')]

In [97]:
available_functions = {
    "get_current_weather": get_current_weather,
}

In [98]:
messages.append(response_message)  # extend conversation with assistant's reply

# 调用知识库

In [104]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    location = function_args.get("location")
    print(f"正在对【{location}】调用知识库……")
    function_response = function_to_call(
        location=location,
    )
    print(f"知识库返回：{function_response}")
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

正在对【北京】调用知识库……
知识库返回：{"location": "北京", "temperature": "-4"}
正在对【北京】调用知识库……
知识库返回：{"location": "北京", "temperature": "-4"}


In [100]:
messages

[{'role': 'user', 'content': '我明天想去北京玩，需要随身携带些什么东西？'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hTxXPG3Q7T4uBuCaK828q7S2', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_TWcj0quTMn8MY3qjPYC1o3UR', function=Function(arguments='{"location": "北京"}', name='get_current_weather'), type='function')]),
 {'tool_call_id': 'call_hTxXPG3Q7T4uBuCaK828q7S2',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "北京", "temperature": "-4"}'},
 {'tool_call_id': 'call_TWcj0quTMn8MY3qjPYC1o3UR',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "北京", "temperature": "-4"}'}]

# 将知识库中返回的信息提交给GPT，再次生成回答

In [101]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)

In [102]:
second_response

ChatCompletion(id='chatcmpl-8Xn95n7aph6QSdyPt65hZD8xvmN9N', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='明天北京的天气预计为-4摄氏度，需要携带保暖的衣物，例如羽绒服、围巾、手套等。另外，还应该携带一个小背包，方便放置必需的物品，如水瓶、地图、手机充电器等。同时，需要携带有效的身份证件和钱包等日常用品。最后，也可以考虑携带一些零食或者稍做准备的午餐，以备不时之需。', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1703064431, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=158, prompt_tokens=126, total_tokens=284))

# 得到知识库信息加成后的最终回答

In [103]:
print(second_response.choices[0].message.content)

明天北京的天气预计为-4摄氏度，需要携带保暖的衣物，例如羽绒服、围巾、手套等。另外，还应该携带一个小背包，方便放置必需的物品，如水瓶、地图、手机充电器等。同时，需要携带有效的身份证件和钱包等日常用品。最后，也可以考虑携带一些零食或者稍做准备的午餐，以备不时之需。
